<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/LogisticEmoInt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from google.colab import  drive
drive.mount('/content/drive')

df_train = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/Data/Train/clean_all_train.csv')
df_test = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/Data/Test/clean_all_test.csv')
df_train = df_train.sample(frac=1)
df_test = df_test.sample(frac=1)
def remove_punct(text):
  text = "".join([char for char in text if char!="'" ])
  clean = re.sub(r"""[,.;@#?<>()""-/!&$]+\ *"""," ",text, flags=re.VERBOSE)
  return clean
def remove_stop(text):
  sw = stopwords.words("english")
  sw.append('br')
  clean = " ".join([x for x in text.split() if x not in sw])
  return clean
df_train['Tweets'] = df_train['Tweets'].apply(lambda tweet : remove_punct(tweet))
df_test['Tweets'] = df_test['Tweets'].apply(lambda tweet : remove_punct(tweet))
df_train['Tweets'] = df_train['Tweets'].apply(lambda tweet : remove_stop(tweet))
df_test['Tweets'] = df_test['Tweets'].apply(lambda tweet : remove_stop(tweet))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
category_dict = {"anger":0,"sadness":1,"fear":2,"joy":3}
vectorizer = CountVectorizer(min_df = 5)
X = vectorizer.fit_transform(df_train['Tweets'])
X = X.toarray()
X = [x/(sum(x)+1) for x in X]
y = [category_dict[x] for x in df_train['Category']]
test_X = vectorizer.transform(df_test['Tweets'])
test_X = test_X.toarray()
test_y = [category_dict[x] for x in df_test['Category']]

In [ ]:
clf = OneVsRestClassifier(LogisticRegression()).fit(X,y)

In [ ]:
clf.score(test_X,test_y)

0.7390197326543603

In [ ]:
clf.predict(vectorizer.transform(['i am very angry and you better do something']))[0]

0

In [ ]:
clf.predict(vectorizer.transform(['too bad so sad']))[0]

1

In [ ]:
clf.predict(vectorizer.transform(['i amd so terrified by crimes going on in the city everyday there is a new murder']))[0]

2

In [ ]:
clf.predict(vectorizer.transform(['very happy best moments of life']))[0]

3